In [3]:
import MetaTrader5 as mt5

mt5.initialize()
symbols = mt5.symbols_get()
for s in symbols:
    print(s.name)
mt5.shutdown()


EURUSD
GBPUSD
USDCHF
USDJPY
USDCAD
AUDUSD
AUDNZD
AUDCAD
AUDCHF
AUDJPY
CHFJPY
EURGBP
EURAUD
EURCHF
EURJPY
EURNZD
EURCAD
GBPCHF
GBPJPY
CADCHF
CADJPY
CHFPLN
EURDKK
EURHUF
EURNOK
EURPLN
EURZAR
GBPAUD
GBPCAD
GBPNZD
GBPPLN
GBPZAR
NZDCAD
NZDCHF
NZDJPY
NZDUSD
USDCNH
USDCZK
USDDKK
USDHKD
USDHUF
USDMXN
USDNGN
USDNOK
USDPLN
USDRUB
USDSEK
USDSGD
USDTRY
USDZAR
ZARJPY
EU50.F
FRA40.F
JP225.F
N25.F
SUI20.F
UK100.F
US100.F
US30.F
US500.F
USDIndex
AUS200
FRA40
JPN225
NETH25
SPA35
SUI20
UK100
USA30
USA500
USA100
UKOIL
USOIL
UKOIL.S
USOIL.S
XAGEUR
XAGUSD
XAUEUR
XAUUSD
Cocoa
Coffee
Copper
Cotton
Palladium
Platinum
Sugar
BTCEUR
BTCUSD
ETHEUR
ETHUSD
LTCEUR
LTCUSD
XRPUSD
EUBUND.F
UKGILT.F
US10YR.F
USDTHB
BCHUSD
EOSUSD
BNBUSD
XMRUSD
XLMUSD
#21stCenturyFox
#AMD
#ASML-US
#Adobe
#Alphabet
#Alphabet-A
#Amazon
#Apple
#Baidu
#Bilibili
#BookingHolding
#Broadcom
#Cisco
#FirstSolar
#Hasbro
#Illumina
#Intel
#LamResearch
#Mattel
#MercadoLibre
#MicrochipTechn
#MicronTechnolo
#Microsoft
#MonsterBeverag
#NVIDIA
#Nasdaq
#Net

True

In [6]:
rates = mt5.copy_rates_from_pos("XAUUSD", mt5.TIMEFRAME_H1, 0, 10)
print(rates)


None


In [7]:
if not mt5.symbol_select("XAUUSD", True):
    print("Failed to select symbol XAUUSD")
else:
    print("Symbol XAUUSD selected")


Failed to select symbol XAUUSD


In [10]:
tick = mt5.symbol_info_tick("XAUUSD")
print("Raw tick data for XAUUSD:", tick)


Raw tick data for XAUUSD: None


In [9]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime, timedelta

# ---------------------
# 1. Account Credentials and MT5 Initialization
# ---------------------
login = 48690778
password = "FX3UU.#XhsVNi_w"
server = "HFMarketsGlobal-Demo"

if not mt5.initialize():
    print(f"MetaTrader 5 initialization failed with error code {mt5.last_error()}")
    exit()

if not mt5.login(login, password, server):
    print(f"Failed to log in to {server}. Check your credentials.")
    mt5.shutdown()
    exit()
else:
    print(f"Logged in successfully to {server}!")

# ---------------------
# 2. Symbols and Their Specific Link Values
# ---------------------
symbol_links = {
    "XAUUSD": 3,
    "BTCUSD": 300,
    "BTCEUR": 300,
    "ETHUSD": 30,
    "XRPUSD": 0.03,
    "XAUEUR": 3,
    "GBPAUD": 0.003,
    "GBPNZD": 0.003,
    "CHFJPY": 0.3,
    "EURJPY": 0.3,
    "USA30": 30,
    "USA100": 30,
    "USOIL": 0.3,
    "UKOIL": 0.3,
    "USDJPY": 0.3,
    "GBPJPY": 0.3,
    "USA500": 3,
    "UK100": 3,
    "CADJPY": 0.3,
    "AUDJPY": 0.3,
    "GBPCHF": 0.003,
    "GBPCAD": 0.003,
    "NZDJPY": 0.3,
    "GBPUSD": 0.003,
    "EURGBP": 0.003
}

symbols = list(symbol_links.keys())

# ---------------------
# 3. Helper Functions
# ---------------------
def get_live_data(symbol):
    """Fetch live tick data for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"No tick data for {symbol}")
        return None
    return {
        "bid": tick.bid,    # Sell price
        "ask": tick.ask,    # Buy price
        "last": tick.last,  # Last traded price
        "time": tick.time
    }

def get_ohlc(symbol, timeframe=mt5.TIMEFRAME_M1, num_bars=100):
    """Fetch historical OHLC data as a DataFrame."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None:
        print(f"No historical data for {symbol}")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_parameters(df, link):
    """
    Calculate parameters needed for the strategy.
    For this example, we use:
      vh: the highest high in the historical data
      vl: the lowest low in the historical data
      yh: the last candle's high (assumed as "yesterday's high")
      yl: the last candle's low (assumed as "yesterday's low")
      ch: the highest close in the historical data (for today, example)
      cl: the lowest close in the historical data (for today, example)
    """
    vh = df['high'].max()
    vl = df['low'].min()
    yh = df['high'].iloc[-1]
    yl = df['low'].iloc[-1]
    ch = df['close'].max()
    cl = df['close'].min()
    return vh, vl, yh, yl, ch, cl, link

def build_progressions(vh, vl, yh, yl, ch, cl, link):
    """Build arithmetic progressions for linking logic."""
    V = (vh - vl + link) / 2
    YV = (yh - yl + link) / 2

    # DS progression: starts at yesterday's high and increases by V
    DS = [yh + i * V for i in range(10)]
    # DB progression: starts at yesterday's low and increases by V
    DB = [yl + i * V for i in range(10)]
    # YVS progression: starts at today's high and decreases by YV
    YVS = [ch - j * YV for j in range(10)]
    # YVB progression: starts at today's low and increases by YV
    YVB = [cl + j * YV for j in range(10)]
    return DS, DB, YVS, YVB, V, YV

def check_link_status(price, DS, YVS, link):
    """
    Check linking status by testing if the difference between any DS and YVS element
    approximates the symbol-specific link value.
    """
    error_margin = link  # adjust tolerance if needed
    for i in range(len(DS)):
        for j in range(len(YVS)):
            if abs((DS[i] - YVS[j]) - link) < error_margin:
                return "Linked"
    return "Unlinked"

# ---------------------
# 4. Main Loop: Process Symbols Continuously
# ---------------------
print("Starting live monitoring for multiple symbols...")

try:
    while True:
        for symbol in symbols:
            # Get the fixed link value for the symbol
            link_value = symbol_links.get(symbol)
            if link_value is None:
                print(f"Link value not defined for {symbol}. Skipping.")
                continue

            # Make sure the symbol is selected
            if not mt5.symbol_select(symbol, True):
                print(f"Symbol {symbol} is not available.")
                continue

            live_data = get_live_data(symbol)
            df = get_ohlc(symbol, mt5.TIMEFRAME_M1, num_bars=100)
            
            if live_data is None or df is None:
                continue

            # Calculate parameters using the symbol-specific link value
            vh, vl, yh, yl, ch, cl, link = calculate_parameters(df, link_value)
            # Build arithmetic progressions
            DS, DB, YVS, YVB, V, YV = build_progressions(vh, vl, yh, yl, ch, cl, link)
            # Get the current live price
            current_price = live_data['last']
            # Determine link status based on our linking logic
            status = check_link_status(current_price, DS, YVS, link)
            
            timestamp = datetime.fromtimestamp(live_data['time'])
            print(f"Symbol: {symbol} | Time: {timestamp} | Price: {current_price:.5f} | Link: {link:.5f} | Status: {status}")

        # Wait a short while before the next update
        time.sleep(10)

except KeyboardInterrupt:
    print("Live monitoring stopped by user.")

finally:
    mt5.shutdown()


Logged in successfully to HFMarketsGlobal-Demo!
Starting live monitoring for multiple symbols...
Symbol: XAUUSD | Time: 2025-02-25 14:28:04 | Price: 0.00000 | Link: 3.00000 | Status: Linked
Symbol: BTCUSD | Time: 2025-02-25 14:28:04 | Price: 0.00000 | Link: 300.00000 | Status: Linked
Symbol: BTCEUR | Time: 2025-02-25 14:28:04 | Price: 0.00000 | Link: 300.00000 | Status: Linked
Symbol: ETHUSD | Time: 2025-02-25 14:28:04 | Price: 0.00000 | Link: 30.00000 | Status: Linked
Symbol: XRPUSD | Time: 2025-02-25 14:28:20 | Price: 0.00000 | Link: 0.03000 | Status: Linked
Symbol: XAUEUR | Time: 2025-02-25 14:28:20 | Price: 0.00000 | Link: 3.00000 | Status: Linked
Symbol: GBPAUD | Time: 2025-02-25 14:28:19 | Price: 0.00000 | Link: 0.00300 | Status: Linked
Symbol: GBPNZD | Time: 2025-02-25 14:28:19 | Price: 0.00000 | Link: 0.00300 | Status: Linked
Symbol: CHFJPY | Time: 2025-02-25 14:28:19 | Price: 0.00000 | Link: 0.30000 | Status: Linked
Symbol: EURJPY | Time: 2025-02-25 14:28:20 | Price: 0.00000 |

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime, timedelta

# ---------------------
# 1. Account Credentials and MT5 Initialization
# ---------------------
login = 48690778
password = "FX3UU.#XhsVNi_w"
server = "HFMarketsGlobal-Demo"

if not mt5.initialize():
    print(f"MetaTrader 5 initialization failed with error code {mt5.last_error()}")
    exit()

if not mt5.login(login, password, server):
    print(f"Failed to log in to {server}. Check your credentials.")
    mt5.shutdown()
    exit()
else:
    print(f"Logged in successfully to {server}!")

# ---------------------
# 2. Symbols and Their Specific Link Values
# ---------------------
symbol_links = {
    "XAUUSD": 3,
    "BTCUSD": 300,
    "BTCEUR": 300,
    "ETHUSD": 30,
    "XRPUSD": 0.03,
    "XAUEUR": 3,
    "GBPAUD": 0.003,
    "GBPNZD": 0.003,
    "CHFJPY": 0.3,
    "EURJPY": 0.3,
    "USA30": 30,
    "USA100": 30,
    "USOIL": 0.3,
    "UKOIL": 0.3,
    "USDJPY": 0.3,
    "GBPJPY": 0.3,
    "USA500": 3,
    "UK100": 3,
    "CADJPY": 0.3,
    "AUDJPY": 0.3,
    "GBPCHF": 0.003,
    "GBPCAD": 0.003,
    "NZDJPY": 0.3,
    "GBPUSD": 0.003,
    "EURGBP": 0.003
}

symbols = list(symbol_links.keys())

# ---------------------
# 3. Helper Functions
# ---------------------
def get_live_data(symbol):
    """Fetch live tick data for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"No tick data for {symbol}. Error code: {mt5.last_error()}")
        return None
    return {
        "bid": tick.bid,    # Sell price
        "ask": tick.ask,    # Buy price
        "last": tick.last,  # Last traded price
        "time": tick.time
    }

def get_ohlc(symbol, timeframe=mt5.TIMEFRAME_M1, num_bars=100):
    """Fetch historical OHLC data as a DataFrame."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None:
        print(f"No historical data for {symbol}. Error code: {mt5.last_error()}")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_parameters(df, link):
    vh = df['high'].max()
    vl = df['low'].min()
    yh = df['high'].iloc[-1]
    yl = df['low'].iloc[-1]
    ch = df['close'].max()
    cl = df['close'].min()
    return vh, vl, yh, yl, ch, cl, link

def build_progressions(vh, vl, yh, yl, ch, cl, link):
    """Build arithmetic progressions for linking logic."""
    V = (vh - vl + link) / 2
    YV = (yh - yl + link) / 2

    # DS progression: starts at yesterday's high and increases by V
    DS = [yh + i * V for i in range(10)]
    # DB progression: starts at yesterday's low and increases by V
    DB = [yl + i * V for i in range(10)]
    # YVS progression: starts at today's high and decreases by YV
    YVS = [ch - j * YV for j in range(10)]
    # YVB progression: starts at today's low and increases by YV
    YVB = [cl + j * YV for j in range(10)]
    return DS, DB, YVS, YVB, V, YV

def check_link_status(price, DS, YVS, link):
    """
    Check linking status by testing if the difference between any DS and YVS element
    approximates the symbol-specific link value.
    """
    error_margin = link  # adjust tolerance if needed
    for i in range(len(DS)):
        for j in range(len(YVS)):
            if abs((DS[i] - YVS[j]) - link) < error_margin:
                return "Linked"
    return "Unlinked"

# ---------------------
# 4. Main Loop: Process Symbols Continuously
# ---------------------
print("Starting live monitoring for multiple symbols...")

try:
    while True:
        for symbol in symbols:
            # Get the fixed link value for the symbol
            link_value = symbol_links.get(symbol)
            if link_value is None:
                print(f"Link value not defined for {symbol}. Skipping.")
                continue

            # Make sure the symbol is selected
            if not mt5.symbol_select(symbol, True):
                print(f"Symbol {symbol} is not available. Error code: {mt5.last_error()}")
                continue

            live_data = get_live_data(symbol)
            df = get_ohlc(symbol, mt5.TIMEFRAME_M1, num_bars=100)
            
            if live_data is None or df is None:
                continue

            # Calculate parameters using the symbol-specific link value
            vh, vl, yh, yl, ch, cl, link = calculate_parameters(df, link_value)
            # Build arithmetic progressions
            DS, DB, YVS, YVB, V, YV = build_progressions(vh, vl, yh, yl, ch, cl, link)
            # Get the current live price
            current_price = live_data['last']
            # Determine link status based on our linking logic
            status = check_link_status(current_price, DS, YVS, link)
            
            timestamp = datetime.fromtimestamp(live_data['time'])
            print(f"Symbol: {symbol} | Time: {timestamp} | Price: {current_price:.5f} | Link: {link:.5f} | Status: {status}")

        # Wait a short while before the next update
        time.sleep(10)

except KeyboardInterrupt:
    print("Live monitoring stopped by user.")

finally:
    mt5.shutdown()

Logged in successfully to HFMarketsGlobal-Demo!
Starting live monitoring for multiple symbols...
Symbol: XAUUSD | Time: 2025-02-25 15:19:41 | Price: 0.00000 | Link: 3.00000 | Status: Linked
Symbol: BTCUSD | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 300.00000 | Status: Linked
Symbol: BTCEUR | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 300.00000 | Status: Linked
Symbol: ETHUSD | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 30.00000 | Status: Linked
Symbol: XRPUSD | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 0.03000 | Status: Linked
Symbol: XAUEUR | Time: 2025-02-25 15:19:41 | Price: 0.00000 | Link: 3.00000 | Status: Linked
Symbol: GBPAUD | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 0.00300 | Status: Linked
Symbol: GBPNZD | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 0.00300 | Status: Linked
Symbol: CHFJPY | Time: 2025-02-25 15:19:42 | Price: 0.00000 | Link: 0.30000 | Status: Linked
Symbol: EURJPY | Time: 2025-02-25 15:19:39 | Price: 0.00000 |